## 실습 [26-1]  
### 1. 실습명 : LSTM으로 문장 생성하기
### 2. 실습 목적 및 설명
* 텐서플로우를 이용하여 LSTM 모델을 구축한다.
* 네이버에서 제공하는 nsmc 영화 리뷰 데이터셋을 이용하여 모델을 훈련한다.
* 훈련된 모델을 통해 문자를 기반으로 새로운 문장을 생성한다.

### 3. 관련 장(챕터) : 26.1.1 LSTM을 이용한 문장 생성
### 4. 코드

In [0]:
"""
텐서플로우를 이용하여 LSTM 모델을 간단하게 구축하고,
네이버 오픈소스 데이터인 nsmc 영화 리뷰 데이터셋을 이용하여 모델을 훈련한다.
훈련된 모델을 통해 문자 기반 생성을 진행한다.
"""
# Colab 환경에서 실행할 경우 빠른 모델 훈련을 위해 런타임 유형을 GPU로 설정하는 것이 좋다.
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os
import time
import tensorflow_datasets as tfds

In [0]:
# tf.keras.utils.get_file 함수를 통해 데이터셋을 불러온다.
# tf.data.TextLineDataset 함수로 데이터를 읽는다. 
url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
text_path = tf.keras.utils.get_file('ratings_train.txt', origin=url)  
ds_file = tf.data.TextLineDataset(text_path)

14630912/14628807 [==============================] - 0s 0us/step


In [0]:
# tfds.features.text.Tokenizer()를 이용하여 입력 텍스트를 토큰화할 수 있다.
tokenizer = tfds.features.text.Tokenizer()

In [0]:
print(tokenizer.tokenize("나는 매일 아침 지하철을 탄다"))

['나는', '매일', '아침', '지하철을', '탄다']


In [0]:
# 데이터셋의 구성을 확인한다.
for sample in ds_file.take(3):
  tokens = tokenizer.tokenize(sample.numpy())
  # 텍스트 부분만 출력한다.
  print(tokens[1:-1])

['document']
['아', '더빙', '진짜', '짜증나네요', '목소리']
['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']


In [0]:
# 데이터셋에서 텍스트 부분만을 리스트에 담아 훈련에 사용할 텍스트 데이터셋을 만든다.
docs = []
with open(text_path, 'r',encoding='utf-8') as f:
  next(f)
  for line in f:
    text = line.split('\t')[1]
    docs.append(text)

print("문장 개수: ",len(docs))
print(docs[:5])

문장 개수:  150000
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']


In [0]:
# 텍스트 데이터셋에 포함되는 모든 고유 문자로 vocab을 구축한다.
whole_text = ' '.join(docs)
vocab = sorted(set(whole_text))
print("고유 문자수: {}개".format(len(vocab)))

고유 문자수: 3004개


In [0]:
# 데이터셋에 포함된 모든 문자를 숫자로 치환하여 임베딩한다. 
# 이를 위해 우선적으로 문자-인덱스 사전을 구축한다.
char2idx = {u:i for i, u in enumerate(vocab)}
idex2char = np.array(vocab)

# 텍스트 데이터셋을 숫자 벡터로 임베딩한다.
text_as_int = np.array([char2idx[c] for c in whole_text])

In [0]:
# 각 문자가 어떤 숫자로 매핑되었는지 확인할 수 있다.
print('{')
for char,_ in zip(char2idx, range(20)):
  print(' {:4s}:{:3d},'.format(repr(char), char2idx[char]))
print(' ...\n}')

{
 ' ' :  0,
 '!' :  1,
 '"' :  2,
 '#' :  3,
 '$' :  4,
 '%' :  5,
 '&' :  6,
 "'" :  7,
 '(' :  8,
 ')' :  9,
 '*' : 10,
 '+' : 11,
 ',' : 12,
 '-' : 13,
 '.' : 14,
 '/' : 15,
 '0' : 16,
 '1' : 17,
 '2' : 18,
 '3' : 19,
 ...
}


In [0]:
# 마찬가지로 데이터셋의 텍스트가 어떻게 숫자로 매핑되었는지 확인할 수 있다.
print ('입력 문장: \n{}\n\n숫자 매핑: \n{}'.format(repr(whole_text[:20]), text_as_int[:20]))

입력 문장: 
'아 더빙.. 진짜 짜증나네요 목소리 '

숫자 매핑: 
[1954    0  974 1593   14   14    0 2327 2342    0 2342 2321  789  844
 2091    0 1387 1757 1312    0]


In [0]:
# 샘플 길이와 epoch 길이를 정하고 훈련 샘플을 만든다.
seq_length = 100
examples_per_epoch = len(whole_text)//seq_length
print(examples_per_epoch)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idex2char[i.numpy()])

54356
아
 
더
빙
.


In [0]:
# 훈련 샘플을 배치 크기로 변환한다.
# 샘플 문장의 다음 문자를 예측하도록 훈련해야 하므로 배치 사이즈는 샘플 길이+1로 설정한다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idex2char[item.numpy()])))

'아 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
' 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반'
'개도 아까움. 원작의 긴장감을 제대로 살려내지못했다. 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족'
'도없다 연기못하는사람만모엿네 액션이 없는데도 재미 있는 몇안되는 영화 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? 걍인피니트가짱이다.진짜짱이다♥'
' 볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~ 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해 담백하고 깔끔해서 좋다. 신문기사로만'


In [0]:
# 학습 샘플에서 입력 텍스트와 타깃 텍스트 부분을 명시하는 함수를 만든다.
# map 메서드를 이용해 해당 함수를 각 배치에 적용한다.
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
# 데이터셋에서 입력 텍스트와 타깃 텍스트가 잘 분리 되었는지 확인한다.
for input_example, target_example in dataset.take(1):
  print('입력 텍스트: ', repr(''.join(idex2char[input_example.numpy()])))
  print('타깃 텍스트: ', repr(''.join(idex2char[target_example.numpy()])))

입력 텍스트:  '아 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조'
타깃 텍스트:  ' 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'


In [0]:
# 모델은 임베딩 된 문자 입력이 들어올 때 다음 문자를 예측해서 출력해야 한다.
# 훈련 샘플을 통해 이를 확인할 수 있다.
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("{:4d}단계".format(i))
  print("입력: {} ({:s})".format(input_idx, idex2char[input_idx]))
  print("예상출력: {} ({:s})".format(target_idx, idex2char[target_idx]))
  

   0단계
입력: 1954 (아)
예상출력: 0 ( )
   1단계
입력: 0 ( )
예상출력: 974 (더)
   2단계
입력: 974 (더)
예상출력: 1593 (빙)
   3단계
입력: 1593 (빙)
예상출력: 14 (.)
   4단계
입력: 14 (.)
예상출력: 14 (.)


In [0]:
# 훈련 epoch 당 다루게 될 시퀀스 데이터를 배치 데이터로 만든다.
# 이때 모델의 일반화 능력을 높이기 위해 데이터셋을 섞는다.
# 매 epoch 당 100개 문자로 이루어진 64개의 데이터를 훈련하게 된다.
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# 모델 설계를 위해 vocab size, embedding dimension, rnn units number를 설정한다.
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
# tf.keras.Sequential을 이용해 구성 층을 이루고 모델을 정의한다.
# tf.keras.layers.Embedding은 문자 데이터를 임베딩 벡터 상에 정수 형태로 매핑하는 임베딩 층이다.
# tf.keras.layers.LSTM은 n개의 rnn_units으로 이루어진 순환 신경망 층이다. LSTM 대신 GRU를 사용할 수도 있다.
# tf.keras.Dense는 vocab_size의 크기를 갖는 출력층으로, 출력 결과는 vocab 내의 문자로 이루어진다.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
          tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
# 훈련을 위해 모델을 빌드한다.
model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [0]:
# 훈련에 앞서 배치 크기와 시퀀스 길이, 어휘 사전 크기를 출력하여 모델이 설정한대로 동작하는지 확인한다.
# 모델은 각 배치당 출력 시퀀스에 대한 문자별 확률 분포를 가진다. 
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_prediction = model(input_example_batch)
  print("배치 크기, 시퀀스 길이, 어휘 사전 크기: ",example_batch_prediction.shape)

배치 크기, 시퀀스 길이, 어휘 사전 크기:  (64, 100, 3004)


In [0]:
# 모델 정보를 확인한다.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           769024    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 3004)          3079100   
Total params: 9,095,100
Trainable params: 9,095,100
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 모델을 통해 실제 예측값을 얻고 문자를 생성하기 위해 출력 시퀀스에 대한 범주형 분포로부터 문자 인덱스를 얻는다.
# tf.random.categorical 함수 첫 번째 인자에 로짓, 두 번째 인자로 예측할 샘플의 개수를 정한다.
sampled_indices = tf.random.categorical(example_batch_prediction[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [0]:
# 각 타임 스텝(time step)에서 다음 문자 인덱스에 대하여 예측할 수 있다.
print(sampled_indices)

[2862  174 1246 2133 1162 2003 1791 1749 2045 2244 2110 1495  990 1378
 1342 2724 1858 2773 2437 2298 1069 1862  638  544  705 2901 2300  764
 2003 1321 1538   29 1491 2110  415  852  266 1036 1581 2617 1545 1012
 1755  855 2724 1798 1805  221 2494 2338  128  829  645  767 1869 1651
  643  934 1141 2911  406 2431  240 1437 2909  730 2031  622 2117 1175
 1403 2970  462  392 2139 1315 2161 2865  784  738 1027 1063 1709 2270
 2441 2147 2881 2137  905 1847 2947  176 2334 1016 2453  136 2700 1328
 1666  479]


In [0]:
# 입력 시퀀스에 대하여 예측된 인덱스를 문자로 표현한다.
# 훈련 전 모델이므로 랜덤한 문자 조합이 예측되었음을 알 수 있다.
print("입력: \n", repr("".join(idex2char[input_example_batch[0]])))
print("\n예측된 다음 문자: \n", repr("".join(idex2char[sampled_indices])))

입력: 
 '이란 괜찮은여배우의 발견..스토리가 아쉽지만 볼만햇다 드라이빙 씬 만큼은 어설퍼 보이지 않는다 재미썽요~~^^ 지루하지 않고 계속 웃었네요.ㅋㅋㅋㅋ 영화보고 드디어 제주도에 가요!'

예측된 다음 문자: 
 '햤。렿윔뜨얹숯셤옆졋웃베덴면매틀싹펼참줫듴쌉금겓껭혼줭끄얹맂봭=법웃演넸ㄻ됴븡쿼뵛돔셸넿틀쉈쉣わ촬짘↘넉긬끈쌜뿅긔닝똠환氣착ニ미화꽦엮귓웜띌무흥要會육릳의헀낌꾿됑듧샷좠창윽헬윙뉨실훨》집돜챙②툥맏쁜間'


In [0]:
# 모델 훈련을 위해 손실함수를 설정한다.
# tf.keras.losses.sparse_categorical_crossentropy 손실함수는 이전 차원의 예측과 교차 적용되기 때문에 이 문제에 적합하다
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_prediction)
print("예측 배열 크기(shape): ", example_batch_prediction.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실: ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 3004)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:  8.007784


In [0]:
# 모델을 컴파일하여 훈련 과정을 설정한다.
# 옵티마이저는 adam optimizer로 하였다.
model.compile(optimizer='adam', loss=loss)

In [0]:
# 체크포인트를 저장할 위치를 설정하고 훈련 가중치를 저장하도록 한다.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [0]:
# 훈련을 실행한다.
# 전체 epoch 값은 빠른 훈련을 위해 10으로 설정하였다.
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
840/840 [==============================] - 77s 92ms/step - loss: 3.7328
Epoch 2/30
840/840 [==============================] - 75s 90ms/step - loss: 3.0356
Epoch 3/30
840/840 [==============================] - 75s 90ms/step - loss: 2.8651
Epoch 4/30
840/840 [==============================] - 76s 90ms/step - loss: 2.7675
Epoch 5/30
840/840 [==============================] - 77s 92ms/step - loss: 2.6986
Epoch 6/30
840/840 [==============================] - 77s 92ms/step - loss: 2.6455
Epoch 7/30
840/840 [==============================] - 76s 91ms/step - loss: 2.6024
Epoch 8/30
840/840 [==============================] - 77s 91ms/step - loss: 2.5666
Epoch 9/30
840/840 [==============================] - 76s 91ms/step - loss: 2.5352
Epoch 10/30
840/840 [==============================] - 77s 91ms/step - loss: 2.5077
Epoch 11/30
840/840 [==============================] - 76s 90ms/step - loss: 2.4829
Epoch 12/30
840/840 [==============================] - 76s 90ms/step - loss: 2.4605
E

In [0]:
# 문장 생성에 앞서 가장 마지막에 저장된 체크포인트를 확인한다. (이는 모델 테스트에 사용된다.)
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
# 문장 생성을 위해 모델을 다시 빌드하고 최신 체크포인트를 복원한다. 
# 이때 예측 단계를 단순화하기 위해 배치 사이즈를 1로 한다.
# RNN의 상태값은 이전 타임 스텝에서 다음 스텝으로 연속적으로 전달되는 방식이므로 모델은 한번 빌드된 고정 크기를 사용한다.
# 다른 배치 크기로 모델을 실행하려면 모델을 다시 빌드하고 체크포인트에서 가중치를 복원해야 한다.
# 체크포인트를 복원
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

In [0]:
# 학습된 모델을 이용하여 텍스트를 생성한다.
def generate_text(model, start_string):

  # 생성할 문자의 수
  num_generate = 500

  # 시작 문자열(입력 문자)을 숫자로 변환하여 벡터화한다.
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 리스트
  text_generated = []

  # temperature값을 통해 생성된 텍스트의 예측 가능성을 설정할 수 있다.
  # 값이 높으면 예측 가능한, 즉 훈련 데이터셋에 가까운 텍스트가 되며,
  # 값이 낮으면 예측이 어려운, 랜덤성이 높은 텍스트가 된다.
  # 여러 번의 실험을 통해 적절한 값을 찾을 수 있다. 
  temperature = 1.0

  # reset_states() 함수는 실행하려는 모델이 이전에 실행한 모델과 무관할 때 사용한다.
  # 예측을 위해 앞서 지정한 배치 사이즈 1의 모델을 빌드하였으므로 이전 훈련 모델을 초기화한다.
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # 배치 차원을 제거한다.
    predictions = tf.squeeze(predictions, 0)

    # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측한다.
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 예측된 단어를 다음 입력으로 모델에 전달한다.
    # 은닉층의 상태값 역시 다음 타임 스텝으로 전달된다.
    input_eval = tf.expand_dims([predicted_id], 0) 
    text_generated.append(idex2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

In [0]:
# 시작 문자열을 설정하고 문장을 생성해본다.
print(generate_text(model, start_string="영화"))

<참고>


> https://www.tensorflow.org/tutorials/text/text_generation

